1. Forked from [A keras prototype (0.21174 on PL)](https://www.kaggle.com/knowledgegrappler/a-keras-prototype-0-21174-on-pl) by [Miha Skalic](https://www.kaggle.com/mihaskalic)
2. Added image visualization of correct and incorrect predictions
3. Added confusion matrix
4. Modified 3rd image from `((x_band1+x_band1)/2)` to `((x_band1+x_band2)/2)`
5. Reduced epochs from 25 to 5 (for testing)

In [22]:
import numpy as np # linear algebra
np.random.seed(666)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [23]:
#Load data
train = pd.read_json("../input/train.json")
test = pd.read_json("../input/test.json")
train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
print("done!")

In [24]:
# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band2)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_train = np.array(train.inc_angle)
y_train = np.array(train["is_iceberg"])

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band2)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_test = np.array(test.inc_angle)


X_train, X_valid, X_angle_train, X_angle_valid, y_train, y_valid = train_test_split(X_train
                    , X_angle_train, y_train, random_state=123, train_size=0.75)

In [25]:
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
    
def get_model():
    bn_model = 0
    p_activation = "elu"
    input_1 = Input(shape=(75, 75, 3), name="X_1")
    input_2 = Input(shape=[1], name="angle")
    
    img_1 = Conv2D(16, kernel_size = (3,3), activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_1 = Conv2D(16, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(128, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = GlobalMaxPooling2D() (img_1)
    
    
    img_2 = Conv2D(128, kernel_size = (3,3), activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_2 = MaxPooling2D((2,2)) (img_2)
    img_2 = Dropout(0.2)(img_2)
    img_2 = GlobalMaxPooling2D() (img_2)
    
    img_concat =  (Concatenate()([img_1, img_2, BatchNormalization(momentum=bn_model)(input_2)]))
    
    dense_ayer = Dropout(0.5) (BatchNormalization(momentum=bn_model) ( Dense(256, activation=p_activation)(img_concat) ))
    dense_ayer = Dropout(0.5) (BatchNormalization(momentum=bn_model) ( Dense(64, activation=p_activation)(dense_ayer) ))
    output = Dense(1, activation="sigmoid")(dense_ayer)
    
    model = Model([input_1,input_2],  output)
    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model
model = get_model()
model.summary()

In [26]:
file_path = "model_weights.hdf5"

model = get_model()
model.fit([X_train, X_angle_train], y_train, epochs=5
          , validation_data=([X_valid, X_angle_valid], y_valid)
         , batch_size=32)

model.save_weights(filepath=file_path)

In [27]:
model.load_weights(filepath=file_path)

print("Train evaluate:")
print(model.evaluate([X_train, X_angle_train], y_train, verbose=1, batch_size=200))
print("####################")
print("watch list evaluate:")
print(model.evaluate([X_valid, X_angle_valid], y_valid, verbose=1, batch_size=200))

In [28]:
print("Visualize trained")
pred_valid_frac = model.predict([X_valid, X_angle_valid], verbose=1, batch_size=200)
pred_valid = pd.DataFrame(pred_valid_frac[:,0], columns=['fpred'])
pred_valid['pred'] = pred_valid.fpred.round(0)
pred_valid['y'] = y_valid
pred_valid['error'] = np.abs(pred_valid.y - pred_valid.fpred)
pred_valid['i'] = pred_valid.index

In [29]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, column
output_notebook()
from bokeh.models import LinearColorMapper, Title
color_mapper = LinearColorMapper(palette="Greys256")

def plot_one(image, error=False):
    f = figure(x_range=(0,1), y_range=(0,1), plot_width=150, plot_height=150, toolbar_location=None)
    f.axis.visible = False
    f.image(image=[image], color_mapper=color_mapper, dh=[1], dw=[1], x=[0], y=[0])
    if error:
        f.add_layout(Title(text="{0:.4f}".format(error), align="center"), "left")
    return f

def plot_row(images, error):
    p = row([plot_one(images[:, :, 0], error), plot_one(images[:, :, 1]), plot_one(images[:, :, 2])])
    return p
    
def plot_rows(df, images):
    p = []
    for i, r in df.iterrows():
        idx = int(r.i)
        p.append(plot_row(images[idx], r.error))
    show(column(p))


In [30]:
print("Icebergs:")
icebergs = pred_valid.loc[pred_valid['y'] == 1, :]
icebergs = icebergs.sort_values(['error'])
icebergs = icebergs.reset_index(drop=True)
print("  Easy:")
iceberg_correct = icebergs.loc[icebergs.y == icebergs.pred, :]
plot_rows(icebergs.head(4), X_valid)

In [31]:
print("Icebergs:")
print("  Barely correct:")
plot_rows(iceberg_correct.tail(4), X_valid)

In [32]:
print("Icebergs:")
print("  Wrong!")
iceberg_wrong = icebergs.loc[icebergs.y != icebergs.pred, :]
plot_rows(iceberg_wrong.tail(4), X_valid)

In [33]:
print("Ships:")
ships = pred_valid.loc[pred_valid['y'] == 0, :]
ships = ships.sort_values(['error'])
ships = ships.reset_index(drop=True)
print("  Easy:")
ship_correct = ships.loc[ships.y == ships.pred, :]
plot_rows(ship_correct.head(4), X_valid)

In [34]:
print("Ships:")
print("  Barely correct:")
plot_rows(ship_correct.tail(4), X_valid)

In [35]:
print("Ships:")
print("  Wrong!")
ship_wrong = ships.loc[ships.y != ships.pred, :]
plot_rows(ship_wrong.tail(4), X_valid)

In [36]:
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
import itertools

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes, rotation=90)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print(cm)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

cm = confusion_matrix(y_valid, pred_valid.pred)
plot_confusion_matrix(cm, ['Ship', 'Iceberg'])

In [37]:
prediction = model.predict([X_test, X_angle_test], verbose=1, batch_size=200)

In [38]:
submission = pd.DataFrame({'id': test["id"], 'is_iceberg': prediction.reshape((prediction.shape[0]))})
submission.head(10)

In [39]:
submission.to_csv("./submission.csv", index=False)